In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import seaborn as sns
from itertools import product
from itertools import combinations_with_replacement
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time
import math
%matplotlib inline

In [2]:
from sklearn.cluster import KMeans, SpectralClustering, AffinityPropagation

In [3]:
from utils import *
%load_ext autoreload
%autoreload 2

In [4]:
gj = pd.read_csv('./gj_df.csv')
# gj = gj[['ifp_id', 'ctt', 'cond', 'training', 'team', 'user_id', 'value', 'fcast_date']]
# gj['fcast_year'] = pd.to_datetime(gj['fcast_date']).dt.year
# gj['fcast_week'] = pd.to_datetime(gj['fcast_date']).dt.week
# gj['ifp_week'] = gj['fcast_year'].map(str) + gj['fcast_week'].map(str) + gj['ifp_id']
# gj = gj.drop('fcast_date', axis=1)
# gj = gj.drop_duplicates()
# gj.to_csv('./gj_df.csv', index=False)

In [5]:
# adult = pd.read_csv('../labels.txt', delimiter='\t', header=0, names=['user_id','website','rating'])
# trec = pd.read_csv('../trec-rf10-crowd/trec-rf10-data.txt', delimiter='\t')
# gj = pd.read_csv('./filled_active_df.csv')

# best_users = trec.groupby('workerID').count().sort_values('docID', ascending=False)[:150].index
# trec = trec[trec['workerID'].isin(best_users)]

# r = pd.Series([2,3,2,3], index=[1,2,0,-2])
# trec['label_bin'] = trec['label'].map(r)
gj.head(10)

,ifp_id,ctt,cond,training,team,user_id,value,fcast_year,fcast_week,ifp_week
0,1244-0,1a,1,a,NaN,51,0.20,2015,3,201531244-0
1,1244-0,1a,1,a,NaN,51,0.20,2015,4,201541244-0
2,1244-0,1a,1,a,NaN,51,0.20,2015,5,201551244-0
3,1244-0,1a,1,a,NaN,51,0.20,2015,6,201561244-0
4,1244-0,1a,1,a,NaN,51,0.20,2015,7,201571244-0
5,1244-0,1a,1,a,NaN,51,0.20,2015,8,201581244-0
6,1244-0,1a,1,a,NaN,51,0.09,2015,8,201581244-0
7,1244-0,1a,1,a,NaN,51,0.09,2015,9,201591244-0
8,1394-0,1a,1,a,NaN,51,0.75,2014,42,2014421394-0
9,1394-0,1a,1,a,NaN,51,0.75,2014,43,2014431394-0


In [6]:
# testframe = create_user_task_ids(adult, 'user_id', 'website', 'rating')
testframe = create_user_task_ids(gj, 'user_id', 'ifp_week', 'value', False, True)
testframe.head()

,ifp_id,ctt,cond,training,team,user_id,value,fcast_year,fcast_week,ifp_week,bin,task_id,uid
0,1244-0,1a,1,a,NaN,51,0.2,2015,3,201531244-0,0.2,0,0
1,1244-0,1a,1,a,NaN,51,0.2,2015,4,201541244-0,0.2,1,0
2,1244-0,1a,1,a,NaN,51,0.2,2015,5,201551244-0,0.2,2,0
3,1244-0,1a,1,a,NaN,51,0.2,2015,6,201561244-0,0.2,3,0
4,1244-0,1a,1,a,NaN,51,0.2,2015,7,201571244-0,0.2,4,0


In [132]:
c = pd.cut(
    testframe['value'],
    [-np.inf, .2, .4, .6, .8, np.inf],
    labels=[2,3,5,7,11]
)
testframe['bin_levels'] = c

In [27]:
testframe = testframe[testframe['uid']<1000]

In [46]:
def batcher(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def split(df):
    train_df, validate_df, test_df = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    return train_df, validate_df, test_df

class Model(nn.Module):
    def __init__(self, users, tasks, k=2):
        super(Model, self).__init__()
        self.user_lut = nn.Embedding(users, k)
        self.task_lut = nn.Embedding(tasks, k)

        self.user_bias = nn.Embedding(users, 1)
        self.task_bias = nn.Embedding(tasks, 1)
        self.global_bias = nn.Parameter(torch.FloatTensor(1))
        
    def forward(self, users, jokes):
        user_vectors = self.user_lut(users)
        task_vectors = self.task_lut(jokes)
        user_bias = self.user_bias(users)
        task_bias = self.task_bias(jokes)

        return torch.bmm(user_vectors.unsqueeze(1),
                         task_vectors.unsqueeze(2)).squeeze() \
                         + user_bias.squeeze() + task_bias.squeeze() + self.global_bias.expand_as(user_bias.squeeze())

def val(df, model):
    crit = nn.MSELoss(size_average=False)
    total_loss = 0.
    total_num = 0
    for batch in batcher(df, 100):
        true_rating = Variable(torch.Tensor(batch.bin.values.astype(float)))
        total_num = total_num + true_rating.size(0)
        users = Variable(torch.LongTensor(batch.uid.values))
        tasks = Variable(torch.LongTensor(batch.task_id.values))
        scores = model.forward(users, tasks)
        total_loss += crit(scores, true_rating).data[0]
    return math.sqrt(total_loss/total_num)


def train(train_iter, val_iter, test_iter, model):
    opt = optim.SGD(model.parameters(), lr=0.1)
    crit = nn.MSELoss()

    print("val:", val(validate_df, model))
    for epochs in range(30):
        avg_loss = 0
        total = 0
        for i,batch in enumerate(batcher(train_df, 100)):
            opt.zero_grad()
            rating = Variable(torch.Tensor(batch.bin.values.astype(float)))
#             print(rating)
            users = Variable(torch.LongTensor(batch.uid.values))
            tasks = Variable(torch.LongTensor(batch.task_id.values))
            scores = model.forward(users, tasks) 
#             + torch.sum(model.user_lut.weight.data.pow(2)) + \
#             torch.sum(model.task_lut.weight.data.pow(2)) + torch.sum(model.user_bias.weight.data.pow(2)) + \
#             torch.sum(model.task_bias.weight.data.pow(2))
            loss = crit(scores, rating)
            #if i % 1000==0:
            #    print (loss.data[0])
            loss.backward()
            avg_loss += loss.data[0]
            total += 1
            opt.step()
        print("train:", math.sqrt(avg_loss / float(total)))
        print("val:", val(validate_df, model))
        print(model.user_bias.weight.data)
    return model.user_lut.weight.data, model.user_bias.weight.data

In [47]:
train_df, validate_df, test_df = split(testframe)
users = len(train_df.uid.unique())
tasks = len(train_df.task_id.unique())
model = Model(users, tasks, k=2)
user_vec, user_bias = train(train_df, validate_df, test_df, model)

val: 2.0344810784346175
train: 1.319820234437903
val: 0.8708446363039286

-0.9573
-0.2501
-0.1474
   ⋮    
 1.3903
-0.1703
-0.0891
[torch.FloatTensor of size 1000x1]

train: 0.6830325764679976
val: 0.547055658469096

-0.5904
-0.1287
-0.1246
   ⋮    
 0.8646
-0.0031
-0.0069
[torch.FloatTensor of size 1000x1]

train: 0.4591340022909762
val: 0.4031205534734756

-0.3352
-0.1165
-0.1055
   ⋮    
 0.6189
 0.0340
 0.0253
[torch.FloatTensor of size 1000x1]

train: 0.35502289407552784
val: 0.33101080138363626

-0.1583
-0.1185
-0.0901
   ⋮    
 0.4998
 0.0422
 0.0366
[torch.FloatTensor of size 1000x1]

train: 0.30200100414043224
val: 0.2922280543258729

-3.5753e-02
-1.2143e-01
-7.7582e-02
     ⋮      
 4.3955e-01
 4.3921e-02
 3.9677e-02
[torch.FloatTensor of size 1000x1]

train: 0.2731683632021344
val: 0.2699788908076739

 4.9113e-02
-1.2379e-01
-6.7401e-02
     ⋮      
 4.0749e-01
 4.4266e-02
 3.9710e-02
[torch.FloatTensor of size 1000x1]

train: 0.2564458419047497
val: 0.25638012865284926

 0.

KeyboardInterrupt: 

In [48]:
user_vec, user_bias = model.user_lut.weight.data, model.user_bias.weight.data

In [73]:
user_features = np.zeros((user_vec.numpy().shape[0], user_vec.numpy().shape[1]+1))
user_features[:, :user_vec.numpy().shape[1]] = user_vec.numpy()
user_features[:, -1] = user_bias.numpy().reshape(-1)

In [75]:
user_features

array([[-0.009719  ,  0.02720528,  0.23188628],
       [ 0.01690728,  0.01131544, -0.13132867],
       [-0.00471632,  0.29508391, -0.0234391 ],
       ..., 
       [ 0.04584001,  0.00829574,  0.35903695],
       [ 0.0071434 ,  0.00954419,  0.0429173 ],
       [ 0.0468646 ,  0.00940172,  0.02431819]])

In [82]:
np.savetxt(testframe[['uid','user_id']].drop_duplicates().values)

array([[     0,     51],
       [     1,     52],
       [     2,     63],
       ..., 
       [   997, 124421],
       [   998, 124432],
       [   999, 124439]])

In [77]:
kmeans = KMeans(n_clusters=200, random_state=0).fit(user_features)
print(kmeans.labels_)

[150 113 159  51  99  60  60 135  60  98 195 185  60 121 101  60 147 110
   8   9  96   7  60 195  60 144 185 164  83 121  98 113  23 172 184 126
 197 132  92  77  29 131  77   0 158  66 104  60 167  51   9 110 197 121
  60 189   8  19  92 142 164   7  68  60  60  30 164 163  90  71 168  34
  57   7  60  60  60  92 154 184 184 126  60 105   9 126  60  34 189  94
 155 189  60  68   9  11  60  60 113  60 126  46   9   9 160 172  83 150
  70 144  60 162  68 173 110 100 104  47   8 175  60 171  68  38 104  60
 164 131 161   9 158  51 126  68 195 146 168 172 106 188 110  60   6 126
  60 191   9  97  77 164 173 176  60  60  64 121  51  51 184  60  92   9
   1   0 180  60 184  68  21 173   9 113   8 154  81 164 164 104 131   9
   9 126 173 182  60 185 103  81 126 126  77 184 121   9 165  90  94 126
 199 104  23 154  50   8  70 189  99 117  60 156  43 131  84 157  60 189
  79 143  92   8 113 126 121  47 114  95 166  73 131   0 106   7  60   9
  68 172  83 197  68   0  78  92 171  34 121 172 15

In [136]:
completed, values, ind = compute_individual_dist(mini_tf, True, False)

In [142]:
start = time.time()
features = np.empty((ind.shape[0], ind.shape[1]**2*ind.shape[0]))
delta_matrices_all = np.empty((ind.shape[0], ind.shape[0], ind.shape[1], ind.shape[1]))
score_matrices_all = np.empty((ind.shape[0], ind.shape[0], ind.shape[1], ind.shape[1]))
for user_index in range(values.shape[0]):
    if user_index%100==0:
        print(user_index)
    #check for full joint distribution or add a prior later
    if np.sum(ind[user_index]==0) > 0:
        continue
    #compute delta matrices with all other users where applicable
    else:
        #create a mask so that other half of tasks can be used later to find score matrix
#         mask = np.random.randint(0,2,values.shape).astype(bool)
        mask = np.ones((values.shape)).astype(bool)
        delta_matrices, t_m_i_1, cluster_img = compute_deltas(user_index, completed, values, ind, mask, False, 20)
        features[user_index,:] = cluster_img.flatten()
        delta_matrices_all[user_index,:,:,:] = delta_matrices
#         score_matrices, t_m_i_2 = compute_deltas(user_index, completed, values, ind, ~mask, True, 20)
#         score_matrices_all[user_index,:,:,:] = score_matrices
#         print(np.sum(t_m_i_1), np.sum(t_m_i_2))
#         if len(np.intersect1d(np.array(np.where(t_m_i_1==True)), np.array(np.where(t_m_i_2==True))))>0:
#             print(np.intersect1d(np.array(np.where(t_m_i_1==True)), np.array(np.where(t_m_i_2==True))))
#             print(regret(score_matrices, delta_matrices, \
#                      np.logical_and((t_m_i_1==True), (t_m_i_2==True))))
print(time.time()-start)

0
100
200
300
400
500
600
700
800
900
127.64959692955017


In [149]:
np.sum(np.isnan(features))

110825

In [151]:
from sklearn.cluster import KMeans

def kmeans_missing(X, n_clusters, max_iter=10):
    """Perform K-Means clustering on data with missing values.

    Args:
      X: An [n_samples, n_features] array of data to cluster.
      n_clusters: Number of clusters to form.
      max_iter: Maximum number of EM iterations to perform.

    Returns:
      labels: An [n_samples] vector of integer labels.
      centroids: An [n_clusters, n_features] array of cluster centroids.
      X_hat: Copy of X with the missing values filled in.
    """

    # Initialize missing values to their column means
    missing = ~np.isfinite(X)
    mu = np.nanmean(X, 0, keepdims=1)
    X_hat = np.where(missing, mu, X)

    for i in range(max_iter):
        print(i)
        if i > 0:
            # initialize KMeans with the previous set of centroids. this is much
            # faster and makes it easier to check convergence (since labels
            # won't be permuted on every iteration), but might be more prone to
            # getting stuck in local minima.
            cls = KMeans(n_clusters, init=prev_centroids)
        else:
            # do multiple random initializations in parallel
            cls = KMeans(n_clusters, n_jobs=-1)

        # perform clustering on the filled-in data
        labels = cls.fit_predict(X_hat)
        centroids = cls.cluster_centers_

        # fill in the missing values based on their cluster centroids
        X_hat[missing] = centroids[labels][missing]

        # when the labels have stopped changing then we have converged
        if i > 0 and np.all(labels == prev_labels):
            break

        prev_labels = labels
        prev_centroids = cls.cluster_centers_

    return labels, centroids, X_hat

In [152]:
labels, centroids, X_hat = kmeans_missing(features, 200, max_iter=5)
print(labels)

0
1


/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:893: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


[197  15  25 129   1  15  37  59  15  21  24  24   1  49 161  15   1  88
 118 107   1   1  15  46   3   8   6  40  11  55  21 107  37 163  15   1
 128 187  49 141  33  87 118 130 163   1 199  15   1  55  15   1  27  96
 107  55  95  77  59   1   2  13  31  15  15   1  38 144 135  98  52  59
 117  98  15  15  15  17 159  31  15  31  57  52  15  15  57  25  53 118
  72  33  15  55  15 184  31 107 107 128   3  73  15  86  15  59   1   1
  40  40  15  52 107  53  31  53   3   1   1 140 189  23  53 141 141  15
  18  99  39  15  91  59  33 107   1  95  28 113 162  23  53 141  59  31
  15  45  31  27  44 130  31  89  57  31  17  11  81  23  31  15  99  31
 125   5  22  57 128  11  22  86 107   1   1 124 158  38  12   1   1  15
 141  53 141 199  15  42   1   4  15  33   3  15  55  15 134  21  53   3
 154 199  59   0  25 193   4  53 130 175  15  53   1  99 106   1  15  37
   7  81 113   1 141 107  28 169 128  93  17  36   1   1   1 194  15 141
   3  81   1  33  52  81  52  33  55  77  37 118 13

In [184]:
def calc_cluster_matrix_dist(labels, delta_matrices_all):
#     delta_matrices_new=np.zeros((len(np.unique(labels)), len(np.unique(labels)), delta_matrices_all[0][0].shape[0], \
#                                                               delta_matrices_all[0][0].shape[1]))
    delta_matrices_new=np.empty(delta_matrices_all.shape)
    for i in np.unique(labels):
        if i%100==0:
            print(i)
        missing = ~np.isfinite(delta_matrices_all)
        mu = np.nanmean(delta_matrices_all, 0, keepdims=0)
        filled = np.where(missing, mu, delta_matrices_all)
        cluster_i = np.average(filled[labels==i], axis=0)
        for j in np.unique(labels): 
            if np.sum(np.sum(np.isnan(cluster_i), (1,2)))>0:
                print(i,j)
                missing = ~np.isfinite(deltas_used)
                mu = np.nanmean(delta_matrices_all, 1, keepdims=1)
                X_hat = np.where(missing, mu, X)
            cluster_j = np.average(cluster_i[labels==j], axis=0)
#             delta_matrices_new[i,j] = cluster_j
            delta_matrices_new[np.ix_(labels==i,labels==j)] = cluster_j
    return delta_matrices_new

In [185]:
delta_matrices_clust = calc_cluster_matrix_dist(labels, delta_matrices_all)

0
100


In [189]:
print(np.average([[pairwise_distances(score_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)]))

0.0776220916645


In [190]:
delta_matrices_clust = calc_cluster_matrix_dist(kmeans.labels_, delta_matrices_all)
print(np.average([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)]))

0
100
0.0770279382517


In [193]:
for k_ in range(2,11):
    model = Model(users, tasks, k=k_)
    user_vec, user_bias = train(train_df, validate_df, test_df, model)
    user_features = np.empty((user_vec.numpy().shape[0], user_vec.numpy().shape[1]+1))
    user_features[:, :user_vec.numpy().shape[1]] = user_vec.numpy()
    user_features[:, :-1] = user_bias.numpy()
    kmeans = KMeans(n_clusters=200, random_state=0).fit(user_features)
    delta_matrices_clust = calc_cluster_matrix_dist(kmeans.labels_, delta_matrices_all)
    avg_dist = np.average([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)])
    max_dist = np.max([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)])
    print("K={} score: {}, {}".format(k_, avg_dist, max_dist))

val: 2.0534805581363393
train: 0.6785000668071375
val: 0.27297596169423294
train: 0.2455079876871621
val: 0.23177292823756387
train: 0.2265714453881669
val: 0.22370148841727236
train: 0.22186856493315193
val: 0.2209202693637123
train: 0.22004835924754415
val: 0.21966143299615062
train: 0.219158649219987
val: 0.21898384552321554
train: 0.21865170748941362
val: 0.21857142395474966
train: 0.21832812664990076
val: 0.2182944864299073
train: 0.21810089579953965
val: 0.2180912764652227
train: 0.21792648978623758
val: 0.21792872818239606
train: 0.21778057300708967
val: 0.2177872638569634
train: 0.2176481001638066
val: 0.21765412888569582
train: 0.21751879636900456
val: 0.21752017037388582
train: 0.21738491183119227
val: 0.2173781772053228
train: 0.217240045089088
val: 0.21722199753704172
train: 0.2170785347842071
val: 0.21704611106295554
train: 0.21689519295379342
val: 0.21684547640761825
train: 0.2166852778300971
val: 0.21661560282260073
train: 0.2164446498244093
val: 0.21635278058664725
trai

/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/numpy/core/_methods.py:116: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:248: RuntimeWarning: invalid value encountered in add
  distances += XX
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:400: RuntimeWarning: overflow encountered in square
  max_iter=max_iter, verbose=verbose)
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:400: RuntimeWarning: invalid value encountered in subtract
  max_iter=max_iter, verbose=verbose)
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:401: RuntimeWarning: overflow encountered in square
  inertia = np.sum((X - centers[labels]) ** 2, dtype=np.float64)


0
K=2 score: 0.07221198567612774
val: 2.2423194358614276
train: 0.7294973150204543
val: 0.2832449913020912
train: 0.25099779028384644
val: 0.2343993986418496
train: 0.22830796611023044
val: 0.22490978366595735
train: 0.2227166322422639
val: 0.22167280479301246
train: 0.22056278936072474
val: 0.22021889376343023
train: 0.21951830141534204
val: 0.21944693638237456
train: 0.21893284925849563
val: 0.21898784123147727
train: 0.218569648081213
val: 0.21869085675493266
train: 0.2183261693932684
val: 0.21848535396232144
train: 0.21815222703885986
val: 0.21833473423500593
train: 0.2180208749890265
val: 0.21821847143287273
train: 0.21791652644402623
val: 0.21812426964257983
train: 0.21782956036958445
val: 0.2180443033035051
train: 0.2177536631414168
val: 0.21797327939565111
train: 0.2176844234318902
val: 0.2179073832868934
train: 0.21761853896283134
val: 0.2178436504149933
train: 0.21755334623548458
val: 0.21777960052437015
train: 0.2174865205487035
val: 0.2177129811916402
train: 0.2174158764936

val: 0.21742152916623586
train: 0.21607492035705536
val: 0.2166321971710068
train: 0.2153028708368665
val: 0.21583684073262258
train: 0.21451089335468704
val: 0.21501712797898398
train: 0.21368868294836912
val: 0.21416744601214494
train: 0.2128369059744627
val: 0.21329213870246314
train: 0.21196399854017556
val: 0.21240182665205143
train: 0.2110821958812203
val: 0.21150927523559532
train: 0.21020378769678388
val: 0.21062607806430914
train: 0.20933873165707428
val: 0.20976095250539348
0
100
K=10 score: 0.07700706690149926


In [217]:
np.sum(np.isnan(user_features))

0

In [220]:
# for k_ in range(2,3):
#     model = Model(users, tasks, k=k_)
#     user_vec, user_bias = train(train_df, validate_df, test_df, model)
#     user_features = np.empty((user_vec.numpy().shape[0], user_vec.numpy().shape[1]+1))
#     user_features[:, :user_vec.numpy().shape[1]] = user_vec.numpy()
#     user_features[:, :-1] = user_bias.numpy()
sc = KMeans(n_clusters=200, random_state=0).fit(user_features)
delta_matrices_clust = calc_cluster_matrix_dist(sc.labels_, delta_matrices_all)
avg_dist = np.average([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)])
max_dist = np.max([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)])
print("K={} score: {}, {}".format(k_, avg_dist, max_dist))

/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/numpy/core/_methods.py:116: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:248: RuntimeWarning: invalid value encountered in add
  distances += XX
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:400: RuntimeWarning: overflow encountered in square
  max_iter=max_iter, verbose=verbose)
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:400: RuntimeWarning: invalid value encountered in subtract
  max_iter=max_iter, verbose=verbose)
/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:401: RuntimeWarning: overflow encountered in square
  inertia = np.sum((X - centers[labels]) ** 2, dtype=np.float64)


0
K=2 score: 0.07221198567612774, 0.7124813399972646


In [212]:
kmeans = AffinityPropagation().fit(user_features)
delta_matrices_clust = calc_cluster_matrix_dist(kmeans.labels_, delta_matrices_all)
avg_dist = np.average([[pairwise_distances(delta_matrices_clust[i,j], delta_matrices_all[i,j]) for i in range(1000)] for j in range(1000)])
print("K={} score: {}".format(k_, avg_dist))

0
100
200
300
400
500
600
700
K=2 score: 0.07887018199302073


In [222]:
sc.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,